# MAT281 - Laboratorio N°08


<a id='p1'></a>
## I.- Problema 01


<img src="https://www.cardrates.com/wp-content/uploads/2020/08/shutterstock_576998230.jpg?1" width="480" height="360" align="center"/>


El conjunto de datos se denomina `creditcard.csv` y consta de varias columnas con información acerca del fraude de tarjetas de crédito, en donde la columna **Class** corresponde a: 0 si no es un fraude y 1 si es un fraude.

En este ejercicio se trabajará el problemas de  clases desbalancedas. Veamos las primeras cinco filas dle conjunto de datos:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
# cargar datos
df = pd.read_csv(os.path.join("data","creditcard.csv"), sep=";")
df.head()

Analicemos el total de fraudes respecto a los casos que nos son fraudes:


In [ ]:
# calcular proporciones
df_count = pd.DataFrame()
df_count["fraude"] =["no","si"]
df_count["total"] = df["Class"].value_counts() 
df_count["porcentaje"] = 100*df_count["total"] /df_count["total"] .sum()

df_count

Se observa que menos del 1% corresponde a registros frudulentos. La pregunta que surgen son:

* ¿ Cómo deben ser el conjunto de entrenamiento y de testeo?
* ¿ Qué modelos ocupar?
* ¿ Qué métricas ocupar?

Por ejemplo, analicemos el modelos de regresión logística y apliquemos el procedimiento estándar:

In [ ]:
# datos 
y = df.Class
X = df.drop('Class', axis=1)

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)


# Creando el modelo
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
 
# predecir
lr_pred = lr.predict(X_test)

# calcular accuracy
accuracy_score(y_test, lr_pred)

En general el modelo tiene un **accuracy** del 99,9%, es decir, un podría suponer que el modelo predice casi perfectamente, pero eso esta lejos de ser así.  Para ver por qué es necesario seguir los siguientes pasos:

### 1. Cambiar la métrica de rendimiento

El primer paso es comparar con distintas métricas, para eso ocupemos las 4 métricas clásicas abordadas en el curso:
* accuracy
* precision
* recall
* f-score

En este punto deberá poner las métricas correspondientes y comentar sus resultados.

In [ ]:
# metrics
y_true =  list(y_test)
y_pred = list(lr.predict(X_test))

print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_test, lr_pred))
print('recall:     ',recall_score(y_test, lr_pred))
print('precision:  ',precision_score(y_test, lr_pred))
print('f-score:    ',f1_score(y_test, lr_pred))
print("")

La matriz calculada por **confusion_matrix** difiere de la vista en clases:

In [ ]:
#help(confusion_matrix)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
mc = confusion_matrix(y_true,y_pred)
print(f'FP',mc[0,0])
print(f'FN',mc[1,0])
print(f'TN',mc[0,1])
print(f'TP',mc[1,1])

disp = ConfusionMatrixDisplay(mc, display_labels = None)

Así, en realidad se tiene que la matriz de confusión dada por la función **confusion_matrix** está rotada en $\pi$ grados con respecto a la vista en clases:

<img src="./images/confusion_matrix.png" width="200" height="200" align="center"/>

Hay una alta exactitud (**accuracy**) en el modelo ya que la cantidad de fraudes del conjunto de datos es más bajo en comparación a la muestra, por lo tanto se obtuvieron mayor cantidad proporcional de verdaderos positivos y negativos al comparar con verdaderos y errores (explicado previamente por el Profesor en el enunciado).

Al tener mayor precisión que exhaustividad (**recall**), se deduce que hay una mayor tasa de falsos positivos a verdaderos positivos que viceversa. Esto sale directamente de la matriz de confusión en la cual el número de falsos negativos duplica a los falsos positivos.

Se comparará el valor **F-score** obtenido con el de otros modelos, aunque a priori, 0.8 es más cercano a 1 que a 0 por lo que se comporta de buena manera. Este indicador corresponde al promedio armónico entre precisión y exhaustividad por lo que, en teoría, ya se realizó el análisis de esta métrica.

In [ ]:
# graficar curva roc

def plot_roc_curve(fpr, tpr):
    plt.figure(figsize=(9,4))
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    
# calcular score AUC

probs = lr.predict_proba(X_test) # predecir probabilidades para X_test
probs_tp = probs[:, 1] # mantener solo las probabilidades de la clase positiva 
       
auc = roc_auc_score(y_test, probs_tp)  # calcular score AUC 

print('AUC: %.2f' % auc)

# calcular curva ROC

fpr, tpr, thresholds = roc_curve(y_test, probs_tp) # obtener curva ROC
lrplot = plot_roc_curve(fpr, tpr)

### 2. Cambiar algoritmo

El segundo paso es comparar con distintos modelos. Debe tener en cuenta que el modelo ocupado resuelva el problema supervisado de clasificación.

En este punto deberá ajustar un modelo de **random forest**, aplicar las métricas y comparar con el modelo de regresión logística.

In [ ]:
# train model

# datos
y = df.Class
X = df.drop('Class', axis=1)

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

# crear modelo
rfc = RandomForestClassifier(max_depth=5, n_estimators=500, max_features=1).fit(X_train, y_train) #algoritmo Random Forest

# predecir
rfc_pred = rfc.predict(X_test)
#rfc =  None # algoritmo random forest

# calcular precision
accuracy_score(y_test, rfc_pred)

In [ ]:
# metrics

y_true =  list(y_test)
y_pred = list(rfc.predict(X_test)) # predicciones con random forest

print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")

La exactitud al usar el modelo **random forest** decrece levemente (0.1%) en comparación a la dada en **logistic regression** ya que disminuyen los verdaderos positivos.

Los falsos negativos aumentaron lo que redujo la exhaustividad aproximadamente en un 15%.

El modelo predijo cero falsos positivos por lo que la precisión es perfecta.

De este modo, el **F-score** disminuye marginalmente al comparar el obtenido para regresión logística.

In [ ]:
# calcular score AUC

probs = rfc.predict_proba(X_test) # predecir probabilidades para X_test
probs_tp = probs[:, 1] # mantener solo las probabilidades de la clase positiva 
       
auc = roc_auc_score(y_test, probs_tp)  # calcular score AUC 

print('AUC: %.2f' % auc)

# calcular curva ROC

fpr, tpr, thresholds = roc_curve(y_test, probs_tp) # obtener curva ROC
plot_roc_curve(fpr, tpr)

### 3. Técnicas de remuestreo: sobremuestreo de clase minoritaria

El tercer paso es ocupar ténicas de remuestreo, pero sobre la clase minoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase minoritaria a la clase mayoritaria.

In [ ]:
from sklearn.utils import resample

# concatenar el conjunto de entrenamiento
X = pd.concat([X_train, y_train], axis=1)

# separar las clases
not_fraud = X[X.Class==0]
fraud = X[X.Class==1]

# remuestrear  clase minoritaria
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# recombinar resultados
upsampled = pd.concat([not_fraud, fraud_upsampled])

# chequear el número de elementos por clases
upsampled.Class.value_counts()

In [ ]:
# datos de entrenamiento sobre-balanceados
y_train = upsampled.Class
X_train = upsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, justifique las ventajas y desventjas de este procedimiento.

In [ ]:
upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train) # algoritmo de regresion logistica

# metrics

y_true =  list(y_test)
y_pred = list(upsampled.predict(X_test))


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")

Se obtiene leves cambios en la exactitud al realizar remuestro (se reduce en un 2%).
Aumenta el número de falsos negativos en orden de magnitud 1 y disminuyen los verdaderos negativos, esto produce un aumento de la exhaustividad pero reduce categóricamente la precisión y en consecuencia el **F-Score** 

In [ ]:
# calcular score AUC

probs = upsampled.predict_proba(X_test) # predecir probabilidades para X_test
probs_tp = probs[:, 1] # mantener solo las probabilidades de la clase positiva 
       
auc = roc_auc_score(y_test, probs_tp)  # calcular score AUC 

print('AUC: %.2f' % auc)

# calcular curva ROC

fpr, tpr, thresholds = roc_curve(y_test, probs_tp) # obtener curva ROC
plot_roc_curve(fpr, tpr)

### 4. Técnicas de remuestreo - Ejemplo de clase mayoritaria

El cuarto paso es ocupar ténicas de remuestreo, pero sobre la clase mayoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase mayoritaria  a la clase minoritaria.

In [ ]:
# remuestreo clase mayoritaria
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27) # reproducible results

# recombinar resultados
downsampled = pd.concat([not_fraud_downsampled, fraud])

# chequear el número de elementos por clases
downsampled.Class.value_counts()

In [ ]:
# datos de entrenamiento sub-balanceados

y_train = downsampled.Class
X_train = downsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, justifique las ventajas y desventjas de este procedimiento.

In [ ]:
undersampled = LogisticRegression(solver='liblinear').fit(X_train, y_train) # algoritmo de regresion logistica # modelo de regresi+on logística

# metrics

y_true =  list(y_test)
y_pred = list(undersampled.predict(X_test))


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")

Al igual que con el sobremuestreo, el número de falsos positivos aumenta en orden de magnitud 1 con respecto al análisis sin muestreo y los falsos negativos se reducen a la mitad, reduciendo la precisión  y aumentando la exhaustividad.

Las métricas son similares a las de sobremuestreo salvo exhaustividad que presenta una pequeña baja porcentual de 4%.

In [ ]:
# calcular score AUC

probs = undersampled.predict_proba(X_test) # predecir probabilidades para X_test
probs_tp = probs[:, 1] # mantener solo las probabilidades de la clase positiva 
       
auc = roc_auc_score(y_test, probs_tp)  # calcular score AUC 

print('AUC: %.2f' % auc)

# calcular curva ROC

fpr, tpr, thresholds = roc_curve(y_test, probs_tp) # obtener curva ROC
plot_roc_curve(fpr, tpr)

### 5. Conclusiones

Para finalizar el laboratorio, debe realizar un análisis comparativo con los disintos resultados obtenidos  en los pasos 1-4. Saque sus propias conclusiones del caso.

Los modelos difieren, en particular se obtiene una mayor precisión al utilizar **random forest** en comparación con **regresión logística** con o sin remuestreo. Al realizar muestreos se incrementa la exhaustividad al realizar predicciones pero se decrementa la exactitud bruscamente. 

Por otro lado, al comparar ambos remuestreos en la regresión logística, se tiene una mayor cantidad de falsos positivos y menor cantidad de falsos negativos al realizar un sobremuestreo en comparación a hacer un remuestreo sobre la clase mayoritaria.

Desde las curvas y puntajes ROC, se puede inferir que **random_forest** es un modelo técnicamente mas adecuado para predecir fraudes, aunque marginalmente: **random forest**  obtuvo un puntaje de 0.96 mientras que **regresion logistica** obtuvo 0.92.

Se puede recomendar utilizar un modelo adecuado dependiendo si se desea maximizar precisión o exhaustividad, o en su defecto utilizar un modelo con métricas mas balanceadas.